In [1]:
from google.colab import drive
drive.mount('/content/drive')
print('------------------------------------------------------Подключено к гугл диску')

!git clone https://github.com/ultralytics/yolov5.git

%cd yolov5/
!pip install -r requirements.txt
print('-----------------------------------------------------Установлены нужные библиотеки')


import zipfile
zip_path = "/content/drive/MyDrive/Car_Plate_Dataset/train.zip"
extract_path = "/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Файл {zip_path} успешно распакован в {extract_path}")


zip_path = "/content/drive/MyDrive/Car_Plate_Dataset/augmented_val.zip"
extract_path = "/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Файл {zip_path} успешно распакован в {extract_path}")

print('---------------------------------------------------Все подготовлено, можно запускать обучение')

Mounted at /content/drive
------------------------------------------------------Подключено к гугл диску
Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.13 MiB | 16.34 MiB/s, done.
Resolving deltas: 100% (11859/11859), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [28]:
import os
import torch
import subprocess

weights_path = '/content/drive/MyDrive/Car_Plate_Dataset/best.pt'  # Путь к весам
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

freeze_layers = 10 # Замораживаем первые 10 слоев
if freeze_layers > 0:
    for i, (name, param) in enumerate(model.named_parameters()):
        if i < freeze_layers:
            param.requires_grad = False
        else:
            break
    print(f"Frozen {freeze_layers} layers")

data_path = '/content/drive/MyDrive/Car_Plate_Dataset/dataset_car.yaml'
image_size = 1024
batch_size = 16
epochs = 30
patience = 10

train_command = [
    'python',
    '/content/yolov5/train.py',
    '--imgsz', str(image_size),
    '--batch', str(batch_size),
    '--epochs', str(epochs),
    '--data', data_path,
    '--weights', weights_path,
    '--cache',
    '--device', '0',
    '--patience', str(patience),
    '--optimizer', 'Adam'
]

print(f"Training command: {' '.join(train_command)}")

process = subprocess.Popen(train_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

print(stdout.decode())
print(stderr.decode())

return_code = process.returncode
print(f"Return code: {return_code}")


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-3-18 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


Frozen 10 layers
Training command: python /content/yolov5/train.py --imgsz 1024 --batch 16 --epochs 30 --data /content/drive/MyDrive/Car_Plate_Dataset/dataset_car.yaml --weights /content/drive/MyDrive/Car_Plate_Dataset/best.pt --cache --device 0 --patience 10 --optimizer Adam

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-18 17:27:17.761357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742318837.782418   19339 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742318837.794807   19339 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registere